In [ ]:
from PIL import Image
import numpy as np
from numpy import asarray

In [ ]:
def rgb_to_hsb(altura, largura, rgb_array):
  hsb_array = rgb_array.copy()
  for x in range(altura):
    for y in range(largura):
      rgb = hsb_array[x][y]
      maximo = np.max(rgb)
      minimo = np.min(rgb)
      r = rgb[0]
      g = rgb[1]
      b = rgb[2]
      v = maximo
      if maximo == minimo:
        h = 0
      elif maximo == r and g>=b:
        h = 60 * (g-b)/(maximo-minimo)
      elif maximo ==r and g<b:
        h = 60 * (g-b)/(maximo-minimo) + 360
      elif maximo == g:
        h = 60 * (b-r)/(maximo-minimo) + 120;
      elif maximo == b:
        h = 60 * (r-g)/(maximo-minimo) + 240;
      if maximo == 0.0:
        s = 0.0
      else:
        s = 1.0 - (minimo / maximo)
      hsb_array[x][y] = [h, s, v]
  return hsb_array

In [ ]:
def hsb_to_rgb(altura, largura, hsb_array_toconvert):
  newrgb_array = hsb_array_toconvert.copy()
  for x in range(altura):
    for y in range(largura):
      hsv = newrgb_array[x][y]
      h = hsv[0]
      s = hsv[1]
      v = hsv[2]
      if s == 0:
        nr = v
        ng = v
        nb = v
      else:
        sector_pos = h/60
        sector_number = int(sector_pos)
        fractional_sector = sector_pos - sector_number

        p = v * (1.0 - s)
        q = v * (1.0 - (s * fractional_sector))
        t = v * (1.0 - (s * (1 - fractional_sector)))

        if sector_number == 0:
            nr = v
            ng = t
            nb = p
        elif sector_number == 1:
            nr = q
            ng = v
            nb = p
        elif sector_number == 2:
            nr = p
            ng = v
            nb = t
        elif sector_number == 3:
            nr = p
            ng = q
            nb = v
        elif sector_number == 4:
            nr = t
            ng = p
            nb = v
        elif sector_number == 5:
            nr = v
            ng = p
            nb = q
        if nr > 1 and ng >1 and nb > 1:
          nr = 1
          ng = 1
          nb = 1
        if nr <= 0 and ng <=0 and nb <= 0:
          nr = 0
          ng = 0
          nb = 0
      newrgb_array[x][y]=[nr*255, ng*255, nb*255]

  return newrgb_array

In [ ]:
def changehue(altura, largura, change, typechange, array_tochange):
  # 0 increase
  # 1 decrease
  newhsb_array = array_tochange.copy()
  for x in range(altura):
    for y in range(largura):
      if typechange == 0:
        if newhsb_array[x][y][0] == 360 or newhsb_array[x][y][0] + change > 360:
          newhsb_array[x][y][0] = 360
        else:
          newhsb_array[x][y][0] = newhsb_array[x][y][0] + change
      if typechange == 1:
        if newhsb_array[x][y][0] <= change:
          newhsb_array[x][y][0] = 0
        else:
          newhsb_array[x][y][0] = newhsb_array[x][y][0] - change

  rgbchanged = hsb_to_rgb(altura, largura, newhsb_array)
  return rgbchanged

In [ ]:
def changesaturation(altura, largura, change, typechange, array_tochange):
  # 0 increase
  # 1 decrease
  newhsb_arrays = array_tochange.copy()
  for x in range(altura):
    for y in range(largura):
      if typechange == 0:
        if newhsb_arrays[x][y][1] == 1 or newhsb_arrays[x][y][1] + change > 1:
          newhsb_arrays[x][y][1] = 1
        else:
          newhsb_arrays[x][y][1] = newhsb_arrays[x][y][1] + change
      if typechange == 1:
        if newhsb_arrays[x][y][1] <= change:
          newhsb_arrays[x][y][1] = 0
        else:
          newhsb_arrays[x][y][1] = newhsb_arrays[x][y][1] - change

  rgbchangeds = hsb_to_rgb(altura, largura, newhsb_arrays)
  return rgbchangeds

In [ ]:
def negativergb(altura, largura, rgb_array):
  negativearrayrgb = rgb_array.copy()
  for x in range(altura):
    for y in range(largura):
      negativearrayrgb[x][y][0] = 255-negativearrayrgb[x][y][0]
      negativearrayrgb[x][y][1] = 255-negativearrayrgb[x][y][1]
      negativearrayrgb[x][y][2] = 255-negativearrayrgb[x][y][2]
  return negativearrayrgb


In [ ]:
def negativehsv(altura, largura, hsv_array):
  negativearrayhsv = hsv_array.copy()
  for x in range(altura):
    for y in range(largura):
      negativearrayhsv[x][y][2] = 1-negativearrayhsv[x][y][2]
  return negativearrayhsv


In [ ]:
def read_configuration(filename):
    configuration = {}
    with open(filename, 'r') as file:
        for line in file:
            key, value = line.strip().split(':')
            key = key.strip()
            value = value.strip()
            if key == "altura":
                configuration[key] = int(value)
                linhas = int(value)
            elif key == "largura" or key == "stride":
                configuration[key] = int(value)
            elif key == "filtro":
                # Lê as linhas seguintes para obter o filtro
                filter_matrix = []
                for _ in range(linhas):
                    filter_row = [float(val) for val in file.readline().strip().split()]
                    filter_matrix.append(filter_row)
                configuration[key] = filter_matrix
    return configuration

In [ ]:
def correlacao(image, filtro, alturaf, larguraf, stride):

    altura, largura, canais = image.shape

    altura_filtro = alturaf
    largura_filtro = larguraf

    desvio_x = largura_filtro // 2
    desvio_y = altura_filtro // 2

    altura_resultado = (altura - altura_filtro) // stride
    largura_resultado = (largura - largura_filtro) // stride

    resultado = np.zeros((altura_resultado, largura_resultado, canais), dtype=np.uint8)

    for y in range(0, altura - altura_filtro, stride):
        for x in range(0, largura - largura_filtro, stride):
            for c in range(canais):
                sum = 0.0
                for fy in range(altura_filtro):
                    for fx in range(largura_filtro):
                        img_x = x + fx - desvio_x
                        img_y = y + fy - desvio_y

                        if img_x >= 0 and img_x < largura and img_y >= 0 and img_y < altura:
                            sum += image[img_y][img_x][c] * filtro[fy][fx]

                resultado_x = x // stride
                resultado_y = y // stride
                resultado[resultado_y][resultado_x][c] = np.clip(int(sum), 0, 255)

    return resultado

In [ ]:
def expansao_histograma(imagem):
    minimo = np.min(imagem)
    maximo = np.max(imagem)

    resultado = np.zeros_like(imagem, dtype=np.uint8)

    altura, largura, canais = imagem.shape

    for y in range(altura):
        for x in range(largura):
          for c in range(canais):
            pixel = imagem[y][x][c]
            # formula da expansão do histograma
            resultado_pixel = (pixel - minimo) / (maximo - minimo) * 255
            resultado[y][x][c] = resultado_pixel

    return resultado

In [ ]:
image = Image.open("DancingInWater.jpg")
img_array = asarray(image)
altura, largura, _ = img_array.shape
normal_array = img_array/255
image

In [ ]:
hsb = rgb_to_hsb(altura, largura, normal_array)
newrgb = hsb_to_rgb(altura, largura, hsb)
img2 = Image.fromarray(newrgb.astype(np.uint8))
img2

In [ ]:
hsb2 = changesaturation(altura, largura, 0.4, 0, hsb)
img3 = Image.fromarray(hsb2.astype(np.uint8))
img3

In [ ]:
hsb3 = changehue(altura, largura, 90, 0, hsb)
img4 = Image.fromarray(hsb3.astype(np.uint8))
img4

In [ ]:
nrgb = negativergb(altura, largura, img_array)
img5 = Image.fromarray(nrgb.astype(np.uint8))
img5

In [ ]:
nhsv = negativehsv(altura, largura, hsb)
img6 = Image.fromarray(nrgb.astype(np.uint8))
img6

In [ ]:
config = read_configuration("filtro.txt")

alturafiltro = config.get("altura")
largurafiltro = config.get("largura")
stride = config.get("stride")
filtro = config.get("filtro")

print(filtro)

[[0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044], [0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044], [0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044], [0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044], [0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044], [0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044], [0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044], [0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044, 0.0044], [0.0044, 0.0044, 0.0044

In [ ]:
img_correlacao = correlacao(img_array, filtro, alturafiltro, largurafiltro, stride)
img_correlacao = Image.fromarray(img_correlacao)
img_correlacao

In [ ]:
img_box_3x3 = img_correlacao
img_box_3x3

In [ ]:
img_box_15x1 = img_correlacao
img_box_15x1

In [ ]:
img_box_15x1_as_array = asarray(img_box_15x1)
img_box_1x15 = correlacao(img_box_15x1_as_array, filtro, alturafiltro, largurafiltro, stride)
img_box_1x15 = Image.fromarray(img_box_1x15)
img_box_1x15

In [ ]:
img_box_15x15 = correlacao(img_array, filtro, alturafiltro, largurafiltro, stride)
img_box_15x15 = Image.fromarray(img_box_15x15)
img_box_15x15

In [ ]:
img_sobel_vertical = correlacao(img_array, filtro, alturafiltro, largurafiltro, stride)
img_sobel_vertical = Image.fromarray(img_sobel_vertical)
img_sobel_vertical

In [ ]:
img_sobel_horizontal = correlacao(img_array, filtro, alturafiltro, largurafiltro, stride)
img_sobel_horizontal = Image.fromarray(img_sobel_horizontal)
img_sobel_horizontal

In [ ]:
img_sobel = asarray(img_sobel_vertical) + asarray(img_sobel_horizontal)
img_sobel = Image.fromarray(img_sobel)
img_sobel

In [ ]:
img_sobel_asarray = asarray(img_sobel)
histograma_sobel = expansao_histograma(img_sobel_asarray)
histograma_sobel = Image.fromarray(histograma_sobel)
histograma_sobel